# Patching Python builtins (third-party library compatibility)

Not every Python library is implemented to accept pathlib-compatible objects like those implemented by cloudpathlib. Many libraries will only accept strings as filepaths. These libraries then may internally use `open`, functions from `os` and `os.path`, or other core library modules like `glob` to navigate paths and manipulate them.

This means that out-of-the-box you can't just pass a `CloudPath` object to any method of function and have it work. For those implemented with `pathlib`, this will work. For anything else the code will throw an exception at some point.

The long-term solution is to ask developers to implement their library to support either (1) pathlib-compatible objects for files and directories, or (2) file-like objects passed directly (e.g., so you could call `CloudPath.open` in your code and pass the the file-like object to the library).

The short-term workaround that will be compatible with some libraries is to patch the builtins to make `open`, `os`, `os.path`, and `glob` work with `CloudPath` objects. Because this overrides default Python functionality, this is not on by default. When patched, these functions will use the `CloudPath` version if they are passed a `CloudPath` and will fallback to their normal implementations otherwise.

These methods can be enabled by setting the following environment variables:
 - `CLOUDPATHLIB_PACTH_ALL=1` - patch all the builtins we implement: `open`, `os` functions, and `glob`
 - `CLOUDPATHLIB_PACTH_OPEN=1` - patch the builtin `open` method
 - `CLOUDPATHLIB_PACTH_OS_FUNCTIONS=1` - patch the `os` functions
 - `CLOUDPATHLIB_PACTH_GLOB=1` - patch the `glob` module

You can set environment variables in many ways, but it is common to either pass it at the command line with something like `CLOUDPATHLIB_PACTH_ALL=1 python my_script.py` or to set it in your Python script with `os.environ['CLOUDPATHLIB_PACTH_ALL'] = 1`. Note, these _must_ be set before any `cloudpathlib` methods are imported.

Alternatively, you can call methods to patch the functions.

```python
from cloudpathlib import patch_open, patch_os_functions, patch_glob

# patch builtins
patch_open()
patch_os_functions()
patch_glob()
```

These patch methods are all context managers, so if you want to control where the patch is active, you can use them in a `with` statement. For example:

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from glob import glob

from cloudpathlib import patch_glob, CloudPath

try:
    glob(CloudPath("s3://ladi/Images/FEMA_CAP/2020/70349/DSC_000*.jpg"))
except Exception as e:
    print("Unpatched version fails:")
    print(e)


with patch_glob():
    print("Patched succeeds:")
    glob(CloudPath("s3://ladi/Images/FEMA_CAP/2020/70349/DSC_000*.jpg"))

    # or equivalently
    glob("DSC_000*.jpg", root_dir=CloudPath("s3://ladi/Images/FEMA_CAP/2020/70349/"))

Unpatched version fails:
'S3Path' object is not subscriptable
Patched succeeds:


ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

In [9]:
list(CloudPath("s3://openfoodfacts-images/").iterdir())

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

In [ ]:
Below we'll show some examples of code that works with the patched versions.

## Patching `open`

Sometimes


### Patching `open` in Jupyter notebooks

Jupyter notebooks have their own version of `open` that is injected into the global namespace of the notebook. This means that you must _additionally_ replace that version of open with the patched version if you want to use `open` in a notebook. This can be done with the `patch_open` method by adding the following to the top of the notebook.

```python
from cloudpathlib import patch_open

# replace jupyter's `open` with one that works with CloudPath
open = patch_open()
```

In [16]:
from cloudpathlib import CloudPath, patch_open

# example of a function within a third-party library
def library_function(filepath: str):
    with open(filepath, "r") as f:
        print(f.read())


# create file to read
cp = CloudPath("s3://cloudpathlib-test-bucket/patching_builtins/file.txt")

# fails with a TypeError if passed a CloudPath
try:
    library_function(cp)
except Exception as e:
    print(e)


[Errno 2] No such file or directory: '/var/folders/sz/c8j64tx91mj0jb0vd1s4wj700000gn/T/tmpvnzs5qnd/cloudpathlib-test-bucket/patching_builtins/file.txt'


In [11]:
from cloudpathlib import CloudPath, patch_open

# jupyter patce
open = patch_open()

# example of a function within a third-party library
def library_function(filepath: str):
    with open(filepath, "r") as f:
        print(f.read())


# create file to read
cp = CloudPath("s3://cloudpathlib-test-bucket/patching_builtins/file.txt")

library_function(cp)


TypeError: ContextDecorator.__call__() takes 2 positional arguments but 3 were given

# `open`

#os

True